---
fast api 테스트

In [ ]:
import requests

query = [
    {"role": "user", "content": "피타고라스의 정리에 대해 알려줘"},
    {"role": "ai", "content": "피타고라스 정리는 직각삼각형에서 빗변의 길이의 제곱이 다른 두 변의 길이의 제곱의 합과 같다는 수학적 정리입니다. 수식으로는 $ c^2 = a^2 + b^2 $로 표현됩니다. 여기서 $ c $는 빗변의 길이, $ a $와 $ b $는 나머지 두 변의 길이를 나타냅니다. 이 정리는 고대 그리스의 수학자 피타고라스의 이름을 따서 명명되었습니다.**결론 요약**  피타고라스 정리는 직각삼각형에서 빗변의 길이 제곱이 나머지 두 변의 길이 제곱의 합과 같다는 수학적 정리입니다. 수식으로는 $ c^2 = a^2 + b^2 $로 표현됩니다.\n**핵심 포인트** - 직각삼각형에서 빗변의 길이 제곱은 다른 두 변의 길이 제곱의 합과 같다.  - 수식은 $ c^2 = a^2 + b^2 $로 표현되며, $ c $는 빗변, $ a $와 $ b $는 나머지 두 변을 나타낸다.  - 이 정리는 고대 그리스의 수학자 피타고라스의 이름을 따 명명되었다."},
    {"role": "user", "content": "지구에 대해 알려줘"},
    {"role": "ai", "content": "지구는 태양계의 세 번째 행성으로, 생명이 존재하는 유일한 알려진 행성입니다. 지구는 약 45억 년 전에 형성되었으며, 액체 물, 대기, 그리고 적절한 온도를 바탕으로 복잡한 생태계를 지속하고 있습니다.### 주요 특징- **지리**: 지구는 바다, 육지, 산맥, 호수, 강 등 다양한 지형으로 구성되어 있습니다.- **대기**: 질소(약 78%), 산소(약 21%)를 주성분으로 하며, 이산화탄소, 아르곤 등이 혼합된 대기를 가지고 있습니다.- **지구의 자전과 공전**: 지구는 약 24시간 주기로 자전하며, 태양을 약 365일 주기로 공전합니다.- **생물 다양성**: 수많은 동물, 식물, 미생물이 지구에서 살고 있으며, 인간은 가장 최근에 등장한 종 중 하나입니다.- **기후**: 다양한 기후대가 존재하며, 기후 변화는 자연적 요인과 인간 활동에 의해 영향을 받고 있습니다.지구는 인간의 생존과 발전에 필수적인 자원과 환경을 제공하지만, 환경 오염, 기후 변화, 생물 다양성 감소 등 여러 문제에 직면해 있습니다."},
    {"role": "user", "content": "방금거 다시 알려줘"},
    ]
payload = {
    "query": query,
}

with requests.post(
    "http://localhost:9508/generateStream",
    json=payload,
    stream=True,
    timeout=(5, 600),
) as resp:
    resp.raise_for_status()
    for chunk in resp.iter_content(chunk_size=4096):
        if chunk:
            print(chunk.decode("utf-8"), end="")


---
local test

In [ ]:
import sys
import os
current_dir = os.getcwd()
sys.path.append(os.path.abspath(os.path.join(current_dir, "..")))
sys.path.append(os.path.abspath(os.path.join(current_dir, "..", "..")))
from supervisor_graph import SupervisorGraph
from langchain_core.messages import HumanMessage, AIMessage
import uuid

In [ ]:
history = [HumanMessage(content='바다에 대해 알려줘')]

supervisor = SupervisorGraph()

inputs = {"messages": history }
request_id = str(uuid.uuid4())
config = {
    'configurable': {
        'thread_id': request_id
    }
}
def format_namespace(namespace):
    return (
        namespace[-1].split(":")[0]
        if len(namespace) > 0
        else "parent graph"
    )
async for namespace, chunk in supervisor.graph.astream(inputs, stream_mode="updates", config=config, subgraphs=True):
        for node_name, node_chunk in chunk.items():
                    print("\n" + "=" * 80)
                    print(f"[🧩 Node: {node_name}]")
                    print(f"[📦 Namespace: {namespace}]")
                    print("=" * 80 + "\n")
                    if node_chunk is not None and "messages" in node_chunk:
                        node_chunk["messages"][-1].pretty_print()
                    else:
                        print("메시지 없음 또는 None 반환됨:")
                        print(node_chunk)


In [ ]:
import uuid
from collections import defaultdict

# ---------- 예시 입력 ----------
history = [
HumanMessage(content='피타고라스의 정리에 대해 알려줘'),
AIMessage(content="피타고라스 정리는 직각삼각형에서 빗변의 길이의 제곱이 다른 두 변의 길이의 제곱의 합과 같다는 수학적 정리입니다. 수식으로는 $ c^2 = a^2 + b^2 $로 표현됩니다. 여기서 $ c $는 빗변의 길이, $ a $와 $ b $는 나머지 두 변의 길이를 나타냅니다. 이 정리는 고대 그리스의 수학자 피타고라스의 이름을 따서 명명되었습니다.**결론 요약**  피타고라스 정리는 직각삼각형에서 빗변의 길이 제곱이 나머지 두 변의 길이 제곱의 합과 같다는 수학적 정리입니다. 수식으로는 $ c^2 = a^2 + b^2 $로 표현됩니다.\n**핵심 포인트** - 직각삼각형에서 빗변의 길이 제곱은 다른 두 변의 길이 제곱의 합과 같다.  - 수식은 $ c^2 = a^2 + b^2 $로 표현되며, $ c $는 빗변, $ a $와 $ b $는 나머지 두 변을 나타낸다.  - 이 정리는 고대 그리스의 수학자 피타고라스의 이름을 따 명명되었다."),
HumanMessage(content='지구에 대해 알려줘'),
AIMessage(content="지구는 태양계의 세 번째 행성으로, 생명이 존재하는 유일한 알려진 행성입니다. 지구는 약 45억 년 전에 형성되었으며, 액체 물, 대기, 그리고 적절한 온도를 바탕으로 복잡한 생태계를 지속하고 있습니다.### 주요 특징- **지리**: 지구는 바다, 육지, 산맥, 호수, 강 등 다양한 지형으로 구성되어 있습니다.- **대기**: 질소(약 78%), 산소(약 21%)를 주성분으로 하며, 이산화탄소, 아르곤 등이 혼합된 대기를 가지고 있습니다.- **지구의 자전과 공전**: 지구는 약 24시간 주기로 자전하며, 태양을 약 365일 주기로 공전합니다.- **생물 다양성**: 수많은 동물, 식물, 미생물이 지구에서 살고 있으며, 인간은 가장 최근에 등장한 종 중 하나입니다.- **기후**: 다양한 기후대가 존재하며, 기후 변화는 자연적 요인과 인간 활동에 의해 영향을 받고 있습니다.지구는 인간의 생존과 발전에 필수적인 자원과 환경을 제공하지만, 환경 오염, 기후 변화, 생물 다양성 감소 등 여러 문제에 직면해 있습니다."),
HumanMessage(content='방금거 다시 알려줘')
  ]
inputs = {
    "messages": history,
}
request_id = str(uuid.uuid4())
config = {"configurable": {"thread_id": request_id}}


FINAL = "[final_answer_node/final_answer_node]"
WORKER_WHITELIST = {
    "[web_research_node/agent]",
    "[history_research_node/history_research_node]"
}

def is_worker(pretty: str) -> bool:
    return pretty in WORKER_WHITELIST

def graph_label_from_meta(meta: dict) -> str:
    ns = meta.get("langgraph_checkpoint_ns") or meta.get("checkpoint_ns") or ""
    left = ns.split("|")[0] if "|" in ns else ns
    return left.split(":")[0] if ":" in left else ""

def instance_id_from_meta(meta: dict, pretty: str) -> str:
    # 같은 타입이 여러 번 등장해도 구분되도록 checkpoint_ns/run_id 기반 인스턴스 키 생성
    ns_full = meta.get("langgraph_checkpoint_ns") or meta.get("checkpoint_ns")
    if ns_full:
        return f"{ns_full}"

# ---------- 상태(인스턴스/타입 기반) ----------
active_id = None                  # 현재 실시간 출력 인스턴스
final_started = False             # FINAL 활성화 여부(시작되면 워커 무시)
final_seen = False                # FINAL 토큰이 실제로 들어왔는지
final_id = None                   # FINAL 인스턴스 id (뒤늦게 와도 셋)

# 워커 타입별 진행 중 인스턴스 집합(inflight)
inflight_by_type = defaultdict(set)     # {type(pretty): set(instance_id)}
seen_worker_types = set()               # 한 번이라도 등장한 워커 타입

# 버퍼/순서/메타
buffers = defaultdict(list)       # 인스턴스별 버퍼
order_ids = []                    # 등장 순서(인스턴스 단위)
id2role = {}                      # 인스턴스 -> "worker"/"final"

def _flush(iid: str, newline: bool = False):
    if buffers[iid]:
        print("".join(buffers[iid]), end="", flush=True)
        buffers[iid].clear()
    if newline:
        print("\n\n--------------------------------------------------------------\n\n", flush=True)

def _activate(iid: str):
    global active_id
    active_id = iid
    _flush(iid, newline=True)

def _all_workers_quiet() -> bool:
    # 등장한 워커 타입들에 대해 inflight 인스턴스가 하나도 없으면 True
    for t in seen_worker_types:
        if inflight_by_type[t]:
            return False
    return True

def _drain_worker_buffers():
    # 남아있는 워커 버퍼를 등장 순서대로 방출(+개행)

    for iid in order_ids:
        if id2role.get(iid) != "worker":
            continue
        if buffers[iid]:
            _activate(iid)
            _flush(iid, newline=True)

def _try_start_final_now():
    """유예 없이: FINAL 토큰을 '이미 봤고(final_seen)', 모든 워커가 조용하면(final quiet) 즉시 FINAL 시작."""
    global final_started
    if final_started or not final_seen or final_id is None:
        return
    if not _all_workers_quiet():
        return
    _drain_worker_buffers()
    _activate(final_id)
    final_started = True

# ---------- 이벤트 루프 ----------
async for event in supervisor.graph.astream_events(inputs, config, version="v2"):
    meta = event.get("metadata") or {}
    node = meta.get("langgraph_node", "") or ""
    label = graph_label_from_meta(meta) or "(unknown)"
    pretty = f"[{label}/{node}]"
    etype = event.get("event")

    # 노드 역할 판별
    if pretty == FINAL:
        role = "final"
    elif is_worker(pretty):
        role = "worker"
    else:
        continue
    iid = instance_id_from_meta(meta, pretty)
    id2role.setdefault(iid, role)
    if iid not in order_ids:
        order_ids.append(iid)

    if role == "final":
        final_id = final_id or iid

    # ----- STREAM -----
    if etype == "on_chat_model_stream":
        chunk = (event.get("data") or {}).get("chunk")
        piece = getattr(chunk, "content", None)
        if not piece:
            continue

        if role == "final":
            final_seen = True
            # FINAL 토큰이 들어왔다고 해서 바로 시작하지 않음.
            # 모든 워커가 조용해야 하고(final quiet), 그때만 시작.
            _try_start_final_now()
            if final_started and active_id == final_id:
                print(piece, end="", flush=True)
            else:
                buffers[iid].append(piece)
            continue

        # ----- 워커 처리 (인스턴스 단위) -----
        seen_worker_types.add(pretty)
        inflight_by_type[pretty].add(iid)

        if final_started:
            # FINAL 시작 후엔 워커 무시(순서 보장)
            continue

        if active_id in (None, iid):
            if active_id is None:
                _activate(iid)
            print(piece, end="", flush=True)
        else:
            buffers[iid].append(piece)

    # ----- END -----
    elif etype == "on_chat_model_end":
        if role == "final":
            if active_id == iid:
                print("", flush=True)
            # final_started는 이미 True일 수도, 아닐 수도 있음(버퍼링만 했을 가능성)
            continue

        # 워커 종료: inflight에서 제거
        if iid in inflight_by_type[pretty]:
            inflight_by_type[pretty].remove(iid)

        if final_started:
            continue

        if active_id == iid:
            print("", flush=True)
            active_id = None

        _try_start_final_now()
